[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# Covid-19 Vaccination Tracking Dashboard

In this assignment I have developed a simple dashboard which downloads and displays Covid-19 data from the UK Health Security Agency (UKHSA)
dashboard. This dashboard presents and compares the number of Covid-19 vaccinations administered during the spring and autumn seasons of 2023 and 2024. It's interesting to see the changing trend in vaccination doses during this period.

In [8]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

In [9]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [10]:
# Initialize the dictionary to store all JSON data
jsondata = {}

# Load each JSON file and store its data in the dictionary with a specific key
with open("spring_doses_2023.json", "rt") as infile:
    jsondata['spring_doses_2023'] = json.load(infile)

with open("spring_doses_2024.json", "rt") as infile:
    jsondata['spring_doses_2024'] = json.load(infile)

with open("autumn_doses_2023.json", "rt") as infile:
    jsondata['autumn_doses_2023'] = json.load(infile)

with open("autumn_doses_2024.json", "rt") as infile:
    jsondata['autumn_doses_2024'] = json.load(infile)

In [11]:
def wrangle_data(rawdata):
    """Parameters: rawdata - data from JSON files or API call. Returns a DataFrame."""
    
    data = {}  # Initialize the dictionary to hold wrangled data
    
    # Iterate through each dataset in rawdata
    for dataset_name, dataset in rawdata.items():  
        for entry in dataset:
            date = entry['date']
            metric = entry['metric']
            value = entry['metric_value']
            
            if date not in data:
                data[date] = {}  # Create a new entry for this date
                
            # Store the value for this metric on this date
            data[date][metric] = value
    
    # Convert the nested dictionary into a DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    
    # Optional: Sort the DataFrame by date
    df.index = pd.to_datetime(df.index)  # Convert date strings to datetime objects
    df.sort_index(inplace=True)
    
    return df  # Return the prepared DataFrame

# Call the function directly on the loaded JSON data
df = wrangle_data(jsondata)

## Refresh the Data

Please click on the button below if you would like to refresh the data shown in the graphs. Please note that the data may appear unchanged as new data is not updated very regularly.

In [12]:
# API access function. This will be called when the button is clicked.
def access_api(button):
    try:
        # Make API calls and check their status
        endpoints = [
            "https://api.ukhsa-dashboard.data.gov.uk/themes/infectious_disease/sub_themes/respiratory/topics/COVID-19/geography_types/Nation/geographies/England/metrics/COVID-19_vaccinations_spring23_dosesByDay",
            "https://api.ukhsa-dashboard.data.gov.uk/themes/infectious_disease/sub_themes/respiratory/topics/COVID-19/geography_types/Nation/geographies/England/metrics/COVID-19_vaccinations_spring24_dosesByDay",
            "https://api.ukhsa-dashboard.data.gov.uk/themes/infectious_disease/sub_themes/respiratory/topics/COVID-19/geography_types/Nation/geographies/England/metrics/COVID-19_vaccinations_autumn23_dosesByDay",
            "https://api.ukhsa-dashboard.data.gov.uk/themes/infectious_disease/sub_themes/respiratory/topics/COVID-19/geography_types/Nation/geographies/England/metrics/COVID-19_vaccinations_autumn24_dosesByDay"
        ]
        
        for url in endpoints:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error if the request fails
            
        # Update button attributes on success
        button.icon = "check"
        button.description = "Data refreshed"
        button.disabled = True
    
    except requests.exceptions.RequestException as e:
        # Handle any request exceptions
        button.icon = "times"
        button.description = "Refresh failed"
        print(f"Error: {e}")

# Create the button
apibutton = wdg.Button(
    description="Refresh data",
    disabled=False,
    button_style="success",  # 'success', 'info', 'warning', 'danger', or ''
    tooltip="Click to refresh the data",
    icon="refresh"  # (FontAwesome names without the `fa-` prefix)
)

# Register the callback function with the button
apibutton.on_click(access_api)

# Display the button
display(apibutton)

Button(button_style='success', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click…

## Vaccination Comparison Graph

This graph represents the different vaccination doses given in the seasons of spring and autumn for the years 2023 and 2024. Select from the list to see a single parameter; to see them all at the same time, select all the parameters in the list provided.

In [13]:
def plot_graph(graph):
    """ Our sample graph plotting function """
    df[graph].plot()
    plt.show() # important! update won't work properly without this

timeseriesdf=pd.read_pickle("timeseriesdf.pkl")

series=wdg.SelectMultiple(
    options=['spring_doses_2023','spring_doses_2024','autumn_doses_2023','autumn_doses_2024'],
    value=['spring_doses_2023','spring_doses_2024','autumn_doses_2023','autumn_doses_2024'],
    rows=4,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

**Author and License** Made by: Muneeza Farooq Qasim (ID: 240221988 ec24780@qmul.ac.uk) Also Acknowledgement of sources and the conditions of the license: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."